## Resource
1. [NLTK Chunking, i.e. Grouping](https://youtu.be/X2vAabgKiuM?t=2165)
2. [NLTK Tokenize]()
3. []()

In [1]:
import re
import pandas as pd
import bs4
import requests
import spacy
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 
import networkx as nx
from tqdm import tqdm


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
faq = pd.read_csv('/content/faq.csv')
faq.shape

(90, 3)

In [3]:
faq.head()

,question,answer,category
0,What if I need to know my correct CGPA?,CGPA is seen on the grade card. The CGPA count...,grade card
1,I have cleared my backlog but the result sheet...,GPA and CGPA are not mentioned on backlog gra...,grade card
2,"I have cleared my backlog, but the result on E...",Grade card shows first attempt GPA and CGPA o...,grade card
3,What do I do if I have not received my grade c...,Grade cards should be collected from the stude...,grade card
4,My name/Father's Name/Mother's name is incorre...,Name correction on grade cards/PDC needs to b...,Corrections on grade card /PDC


In [6]:
text_a = faq.answer[1]
text_q = faq.question[1]

In [8]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize

sentences = sent_tokenize(text_q)
stemmer = PorterStemmer()
new_sentence = []
for i in range(len(sentences)):
    words = word_tokenize(sentences[i])
    words = [stemmer.stem(word) for word in words]
    new_sentence.append(' '.join(words))
print(new_sentence)

['I have clear my backlog but the result sheet doe not show gpa and cgpa .', 'whi ?']


In [ ]:
!python -m -q spacy download en_core_web_lg
!python -m -q spacy download en_core_web_sm
!python -m -q spacy download en

     |████████████████████████████████| 827.9 MB 1.2 MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-py3-none-any.whl size=829180942 sha256=b5348d9269f52fa9887eda99d81e7bd7ac6260b0ccc81dec01b5bec960e6bd9e
  Stored in directory: /tmp/pip-ephem-wheel-cache-p4849i4g/wheels/11/95/ba/2c36cc368c0bd339b44a791c2c1881a1fb714b78c29a4cb8f5
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
     |████████████████████████████████| 12.0 MB 8.4 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


### Keyword Extraction 
[source](https://towardsdatascience.com/keyword-extraction-process-in-python-with-natural-language-processing-nlp-d769a9069d5c)
1. Using NLTK
2. Yakes 

In [ ]:
import spacy 
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
print(doc)
print(doc.ents)

 GPA and CGPA are not mentioned on backlog grade cards. GPA, CGPA will be highlighted on regular trimester grade card only.
(GPA, CGPA, GPA, CGPA)


In [ ]:
!pip install -q git+https://github.com/LIAAD/yake
# Github: https://github.com/LIAAD/yake

import yake
kw_extract = yake.KeywordExtractor()
language = 'en'
max_ngram_size = 3
deduplication_threshold = 0.9
numOfKeywords = 20
custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_threshold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords(text_q)
print(text_q)
for kw in keywords:
  print(kw)

I have cleared my backlog but the result sheet does not show GPA and CGPA. Why?
('GPA and CGPA', 0.01560470695632768)
('show GPA', 0.0605689221549672)
('CGPA', 0.09705179139403544)
('cleared my backlog', 0.1495930091794576)
('result sheet', 0.1495930091794576)
('GPA', 0.15831692877998726)
('cleared', 0.36073110501666333)
('backlog', 0.36073110501666333)
('result', 0.36073110501666333)
('sheet', 0.36073110501666333)
('show', 0.36073110501666333)


In [ ]:
from nltk import word_tokenize, pos_tag
new_tok = pos_tag(word_tokenize(text_q))
new_tok

[('I', 'PRP'),
 ('have', 'VBP'),
 ('cleared', 'VBN'),
 ('my', 'PRP$'),
 ('backlog', 'NN'),
 ('but', 'CC'),
 ('the', 'DT'),
 ('result', 'NN'),
 ('sheet', 'NN'),
 ('does', 'VBZ'),
 ('not', 'RB'),
 ('show', 'VB'),
 ('GPA', 'NNP'),
 ('and', 'CC'),
 ('CGPA', 'NNP'),
 ('.', '.'),
 ('Why', 'WRB'),
 ('?', '.')]

In [ ]:
grammar_np = r'NP: {<DT>?<JJ>*<NN>}'

In [ ]:
chunk_parser = nltk.RegexpParser(grammar_np)
chunk_result = chunk_parser.parse(new_tok)
chunk_result

ModuleNotFoundError: ignored

Tree('S', [('I', 'PRP'), ('have', 'VBP'), ('cleared', 'VBN'), ('my', 'PRP$'), Tree('NP', [('backlog', 'NN')]), ('but', 'CC'), Tree('NP', [('the', 'DT'), ('result', 'NN')]), Tree('NP', [('sheet', 'NN')]), ('does', 'VBZ'), ('not', 'RB'), ('show', 'VB'), ('GPA', 'NNP'), ('and', 'CC'), ('CGPA', 'NNP'), ('.', '.'), ('Why', 'WRB'), ('?', '.')])

In [ ]:
doc = nlp(faq['question'][3])
for tok in doc: 
  print(tok.text, '---', tok.dep_)

What --- dobj
do --- aux
I --- nsubj
do --- ROOT
if --- mark
I --- nsubj
have --- aux
not --- neg
received --- advcl
my --- poss
grade --- compound
card --- dobj
? --- punct


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
t = word_tokenize(faq.question[1])    # all words
t_sw = [word for word in t if not word in stopwords.words()]

pun = ['.', ',', '?']
t = [word for word in t_sw if word != pun]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
print(t)
# print(t_sw)

['I', 'cleared', 'backlog', 'result', 'sheet', 'show', 'GPA', 'CGPA', '.', 'Why', '?']


In [ ]:
len(t)

18

In [ ]:
print(t)

['I', 'have', 'cleared', 'my', 'backlog', 'but', 'the', 'result', 'sheet', 'does', 'not', 'show', 'GPA', 'and', 'CGPA', '.', 'Why']


In [ ]:
# pun = [',', '.', '?']
# for i in pun:
t.remove('?')
len(t)

17

In [ ]:
st = []
for tok in nlp(faq.question[1]):
  if tok.dep_ != 'punct':
    st.append(tok)

In [ ]:
st

[I,
 have,
 cleared,
 my,
 backlog,
 but,
 the,
 result,
 sheet,
 does,
 not,
 show,
 GPA,
 and,
 CGPA,
 Why]

In [ ]:
def get_entities(sent):
  ## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""
  
  for tok in nlp(sent):
    # chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text

  return [ent1.strip(), ent2.strip()]

**Chunk 1**

Defined a few empty variables in this chunk. prv_tok_dep and prv_tok_text will hold the dependency tag of the previous word in the sentence and that previous word itself, respectively. prefix and modifier will hold the text that is associated with the subject or the object.

**Chunk 2**

Next, we will loop through the tokens in the sentence. We will first check if the token is a punctuation mark or not. If yes, then we will ignore it and move on to the next token. If the token is a part of a compound word (dependency tag = “compound”), we will keep it in the prefix variable. A compound word is a combination of multiple words linked to form a word with a new meaning (example – “Football Stadium”, “animal lover”).

As and when we come across a subject or an object in the sentence, we will add this prefix to it. We will do the same thing with the modifier words, such as “nice shirt”, “big house”, etc.

**Chunk 3**

Here, if the token is the subject, then it will be captured as the first entity in the ent1 variable. Variables such as prefix, modifier, prv_tok_dep, and prv_tok_text will be reset.

**Chunk 4**

Here, if the token is the object, then it will be captured as the second entity in the ent2 variable. Variables such as prefix, modifier, prv_tok_dep, and prv_tok_text will again be reset.

**Chunk 5**

Once we have captured the subject and the object in the sentence, we will update the previous token and its dependency tag.

Let’s test this function on a sentence:

In [ ]:
for i in faq.question.head(10):
  print(i)
  print(get_entities(i))
  print('---')

What if I need to know my correct CGPA?
['I', 'correct  CGPA']
---
I have cleared my backlog but the result sheet does not show GPA and CGPA. Why?
['result sheet', 'GPA']
---
I have cleared my backlog, but the result on ERP is not updated .It still shows earlier CGPA.As a result, it is affecting my CGPA of next trimesters also. How do I get it corrected?
['it', 'next  trimesters']
---
What do I do if I have not received my grade card?
['I', 'grade card']
---
My name/Father's Name/Mother's name is incorrect on grade card. How do I get it corrected?
['it', 'grade card']
---
How can I get a new copy of corrected grade cards?
['How  I', 'corrected grade cards']
---
From where should I collect the corrected grade cards/PDC?
['where  I', 'grade cards grade PDC']
---
In how many days will I get the corrected grade cards?
['days  I', 'corrected grade cards']
---
Do I need to submit an Exam form for the end term exam?
['I', 'end term exam']
---
I have an internal backlog, do I need to submit an

In [ ]:
entity_pairs = []

for i in faq.question.head(10):
  entity_pairs.append(get_entities(i))


In [ ]:
entity_pairs

[['I', 'correct  CGPA'],
 ['result sheet', 'GPA'],
 ['it', 'next  trimesters'],
 ['I', 'grade card'],
 ['it', 'grade card'],
 ['How  I', 'corrected grade cards'],
 ['where  I', 'grade cards grade PDC'],
 ['days  I', 'corrected grade cards'],
 ['I', 'end term exam'],
 ['internal  I', 'exam form']]

In [ ]:
def get_relation(sent):

  doc = nlp(sent)

  # Matcher class object 
  matcher = Matcher(nlp.vocab)

  #define the pattern 
  pattern = [{'DEP':'ROOT'}, 
            {'DEP':'prep','OP':"?"},
            {'DEP':'agent','OP':"?"},  
            {'POS':'ADJ','OP':"?"}] 

  matcher.add("matching_1", None, pattern) 

  matches = matcher(doc)
  k = len(matches) - 1

  span = doc[matches[k][1]:matches[k][2]] 

  return(span.text)

In [ ]:
for i in faq.question.head(10):
  print(get_relation(i))

What
Why
get
do
get
get
collect
get
need
need


In [ ]:
relations = [get_relation(i) for i in tqdm(faq.question.head(10))]

100%|██████████| 10/10 [00:00<00:00, 59.79it/s]


In [ ]:
pd.Series(relations).value_counts()

get        4
need       2
What       1
Why        1
do         1
collect    1
dtype: int64